In [ ]:
import matplotlib.pylab as plt
import numpy as np

import sys

import datetime as dt
import pandas as pd

import seaborn as sns

In [ ]:


################################################################################
def sort_courses(courses):

    depts = []
    numbers = []
    unique_courses = []

    for course in courses:
        depts.append(course['id'].split()[0])
        numbers.append(int(course['id'].split()[1]))
        
        identifier = (course['id'],course['type'])
        if identifier not in unique_courses:
            unique_courses.append(identifier)

    idx = np.argsort(numbers)

    #print(idx)

    sorted_courses = []
    for i in idx:
        #print(courses[i])
        sorted_courses.append(courses[i])

    #print(unique_courses)
    unique_courses.sort()
    #print(unique_courses)

    new_courses = []
    for i,c in enumerate(unique_courses):

        new_courses.append({'identifier':c,'instances':[]})

        for course in sorted_courses:
            
            identifier = (course['id'],course['type'])
            if identifier == c:
                new_courses[i]['instances'].append(course)


    return new_courses

################################################################################

################################################################################
def term2date(term):

    # By hand
    #year = 2000 + int(term[1:])
    # Read in
    year = int(term.split()[-1])
    monthstart = 1
    monthend = 5
    daystart = 15
    dayend = 15
    # By hand
    #if term[0] == 'F':
    # Read in
    if term.find('Fall')>=0:
        monthstart = 9
        monthend = 12
        daystart = 1
        dayend = 1
    #day = 15
    # By hand
    #year = 2000 + int(term[1:])
    # Read in
    year = int(term.split()[-1])
    start = dt.datetime(year, monthstart, daystart, 0, 0)
    end = dt.datetime(year, monthend, dayend, 0, 0)

    return start,end

################################################################################




In [ ]:
################################################################################
# Reading in from what we download from Banner
################################################################################
df = pd.read_html('courses_taught_downloads.html')
df = df[-1]
#df['name'] = df['Course Title']
#df['id'] = df['Course']
#df['term'] = df['Associated Term']
courses = []
nc = len(df)
for i in range(nc):
    course = {}
    course['id'] = df['Course'].iloc[i]
    course['name'] = df['Course Title'].iloc[i]
    course['term'] = df['Associated Term'].iloc[i]
    course['year'] = int(df['Associated Term'].iloc[i].split()[-1])
    course['nstudents'] = 18
    if df['Course Title'].iloc[i].find('Lab')>=0:
        course['type'] = 1
    elif df['Course'].iloc[i].find('499')>=0:
        course['type'] = 2
    else:
        course['type'] = 0

    if course['year'] > 2015:
        continue
    courses.append(course)



courses = sort_courses(courses)

#print(courses)

fig, ax = plt.subplots(figsize=(18,12))
course_names = []
for i,course in enumerate(courses):
    #identifier = course['identifier'][0]
    identifier = '{0} - {1}'.format(course['instances'][0]['name'],course['identifier'][0])
    course_names.append(identifier)
    #print(identifier)
    for instance in course['instances']:
        start,end = term2date(instance['term'])
        xranges = [(start,end-start)]
        yrange = (i,1.0)
        # Plot the broken horizontal bars
        #print(xranges)
        fc = 'blue'
        if instance['type']==1:
            fc = 'orange'
        elif instance['type']==2:
            fc = 'red'
        plt.broken_barh(xranges, yrange, facecolors=fc)

#course_names.reverse()
#print(course_names)
y_pos = np.arange(0,len(courses),1) + 0.5
ax.set_yticks(y_pos)
ax.set_yticklabels(course_names)
plt.grid(axis='y')

plt.xlim(dt.datetime(2012,1,1,1,1))

sabbatical_start = dt.datetime(2018,9,1)
sabbatical_end = dt.datetime(2019,9,1)

plt.plot([sabbatical_end,sabbatical_end],[0,y_pos[-1]+3],'k--',label='End of sabbatical')
plt.plot([sabbatical_start,sabbatical_start],[0,y_pos[-1]+3],'k-',label='Start of sabbatical')



ax.xaxis_date()
plt.xticks(fontsize=18)

plt.legend(fontsize=24)

plt.tight_layout()

########################################################



In [ ]:
df

In [ ]:
#courses

In [ ]:
df.sort_values(by='Associated Term')

In [ ]:
# Add some columns
#df.columns
year = []
term = []
start = []
end = []
lab = []

ncourses = len(df)

for i in range(ncourses):
    aterm = df.iloc[i]['Associated Term']
    s,e = term2date(aterm)
    #print(start,end)
    year.append(s.year)
    term.append(aterm.split()[0])
    start.append(s)
    end.append(e)
    
    if (df.iloc[i]['Course Title'].find('Lab')>=0 or df.iloc[i]['Course Title'].find('lab')>=0 or \
    df.iloc[i]['Course Title'].find('Capstone')>=0):
        lab.append('Lab')
    elif df.iloc[i]['Course'].find('499')>=0 or df.iloc[i]['Course'].find('497')>=0 or \
    df.iloc[i]['Course'].find('HNRS')>=0:
        lab.append("IS")
    else:
        lab.append('Lecture')
        
    # Take care of a few things by hand
    if (df.iloc[i]['Associated Term'].find('Spring 2018')>=0) & (df.iloc[i]['Course'].find('PHYS 440')>=0) |\
       (df.iloc[i]['Associated Term'].find('Summer 2018')>=0) & (df.iloc[i]['Course'].find('PHYS 400')>=0) |\
       (df.iloc[i]['Associated Term'].find('Spring 2017')>=0) & (df.iloc[i]['Course'].find('PHYS 400')>=0):
        lab[i] = 'Tutorial'
    
        

        
df['year'] = year
df['term'] = term
df['start'] = start
df['end'] = end
df['type'] = lab


df

In [ ]:
#df['independent study']
df.columns

In [ ]:
# Replace some names with other names
#mask = df['Course Title'] == 'Gen Physics IA Lect'
#df[mask]['Course Title'] = 'Gen Physics IA'

df['Course Title'].replace('Gen Physics IA Lect', 'Gen Physics IA', inplace=True)
df['Course Title'].replace('FRES:Intro EDA & Visualization', 'Intro to Exp. Data Analysis and Viz.', inplace=True)
df['Course Title'].replace('FRESH: Intro to EDA & Visual', 'Intro to Exp. Data Analysis and Viz.', inplace=True)
df['Course Title'].replace('FRSH Intro EDA & Visualization', 'Intro to Exp. Data Analysis and Viz.', inplace=True)

df['Course Title'].replace('Gen Physics IA-Lab', 'General Physics IA Lab', inplace=True)
df['Course Title'].replace('Nuclear & Particle Physics', 'Nuclear and Particle Physics', inplace=True)
df['Course Title'].replace('Software tools for Physicists', 'Software Tools for Physicists', inplace=True)
df['Course Title'].replace('Gen Physics IIA Lab', 'General Physics IIA Lab', inplace=True)
df['Course Title'].replace('Gen Physics IIA', 'General Physics IIA', inplace=True)

df['Course Title'].replace('General Physics I-Lab', 'General Physics I Lab', inplace=True)
df['Course Title'].replace('General Physics II-Lab', 'General Physics II Lab', inplace=True)
df['Course Title'].replace('Modern Physics - Lab', 'Modern Physics Lab', inplace=True)
df['Course Title'].replace('Computational Physics-Lab', 'Intro to Computational Physics Lab', inplace=True)

df['Course Title'].replace('Advanced Laboratory I', 'Adv. Lab/Capstone I', inplace=True)
df['Course Title'].replace('Capstone Research I', 'Adv. Lab/Capstone I', inplace=True)
df['Course Title'].replace('Advanced Laboratory II', 'Adv. Lab/Capstone II', inplace=True)



# CSIS 106 is cross-listed with SCDV 110
df.drop(df[df['Course'] == 'CSIS 106'].index, inplace = True)



In [ ]:
df


In [ ]:
mask = df['type'] != 'IS'
ser = df[mask].sort_values('Course')[['Course','Course Title']].value_counts()
#df[['type']].value_counts()

#type(dftmp)

ser.sort_index()


In [ ]:
# Add a column for "counts" to so we can can see if multiple sections were taught.
df['counts'] = df.groupby(['Associated Term', 'Course Title'])['CRN'].transform('count')

In [ ]:
mask = df['counts']>1

df[mask]

In [ ]:
nrows = len(df)

plt.figure(figsize=(12,12))

for i in range(nrows):
    x = df.sort_values('Course', ascending=False).iloc[i]
    #print(x)
    start = x['start']
    end = x['end']
    #y = f"{x['Course']} - {x['Course Title']}"
    y = f"{x['Course Title']} - {x['Course']}"

    t = x['type']

    linewidth=5
    fmt = 'o-'
    color = 'blue'
    if t=='Lecture':
        color='blue'
    elif t=='Lab':
        color='red'
        fmt = '*-'
    elif t=='IS':
        color='black'
        fmt = 's-'
    elif t=='Tutorial':
        color='black'
        fmt = ':'
        linewidth=4
    
    if x['Instructional Method'] == 'Online asynchronous' or \
    (x['Instructional Method'] == 'Online' and x['Course']=='PHYS 015'):
        color='green'
        linewidth=9
        
    counts = x['counts']
    
    # Skip over the independent studies or the plot is too busy
    if t == 'IS':
        continue

    if counts>1 and (x['Course'] in ['CSIS 200', 'PHYS 110', 'PHYS 142']):
        linewidth=2
        
    # Plot multiple versions if more than one class was taught
    # Get the numerical value of the y-points text
    pts = plt.plot([start,end],[y,y],fmt,linewidth=linewidth, color=color, markersize=1)

    # We only need to plot multiple sections for some classes. Others appear twice or more if
    # some students are taking it as a tutorial at the same time
    if (x['Course'] in ['CSIS 200', 'PHYS 110', 'PHYS 142']):
        if counts==2:
            ypt = pts[0].convert_yunits(y)[0]

            plt.plot([start,end],[ypt+0.25,ypt+0.25],fmt,linewidth=linewidth, color=color, markersize=1)

        elif counts==3:
            ypt = pts[0].convert_yunits(y)[0]

            plt.plot([start,end],[ypt+0.25,ypt+0.25],fmt,linewidth=linewidth, color=color, markersize=1)
            plt.plot([start,end],[ypt-0.25,ypt-0.25],fmt,linewidth=linewidth, color=color, markersize=1)
    
    
plt.xlim(pd.Timestamp('2012-01-01'), pd.Timestamp('2024-09-01')) 
#plt.xlim(pd.Timestamp('2012-01-01'), pd.Timestamp('2016-01-01')) 

#from matplotlib.ticker import AutoMinorLocator
#plt.gca().minorticks_on()
#plt.gca().xaxis.set_minor_locator(AutoMinorLocator(4))
plt.grid(axis='both')#,which='minor')


# Sabbatical
plt.plot([pd.Timestamp('2018-06-15'),pd.Timestamp('2018-06-15')],[0,26],'k--')
plt.plot([pd.Timestamp('2019-09-01'),pd.Timestamp('2019-09-01')],[0,26],'k--')
plt.gca().text(pd.Timestamp('2019-01-01'), 10, r'SABBATICAL', fontsize=23, rotation=90)


###########################################################################################
# Custom legend
# https://matplotlib.org/stable/gallery/text_labels_and_annotations/custom_legends.html

custom_lines = [plt.Line2D([0], [0], color='blue', lw=8),
                plt.Line2D([0], [0], color='red', lw=8),
                plt.Line2D([0], [0], color='black', lw=8, linestyle=':'),
                plt.Line2D([0], [0], color='green', lw=12),
               ]
  
plt.legend(custom_lines,['Lecture', 'Lab','Tutorial', 'Online asynchronous'], fontsize=18, loc='lower left')

plt.ylim(-5)

plt.tight_layout()

plt.savefig('graph_of_courses_taught.png')

In [ ]:
nrows = len(df)

plt.figure(figsize=(12,12))

for i in range(nrows):
    x = df.sort_values('Course', ascending=False).iloc[i]
    #print(x)
    start = x['start']
    end = x['end']
    #y = f"{x['Course']} - {x['Course Title']}"
    y = f"{x['Course Title']} - {x['Course']}"

    t = x['type']

    linewidth=5
    fmt = 'o-'
    color = 'blue'
    if t=='Lecture':
        color='blue'
    elif t=='Lab':
        color='red'
        fmt = '*-'
    elif t=='IS':
        color='black'
        fmt = 's-'
    elif t=='Tutorial':
        color='black'
        fmt = ':'
        linewidth=4
    
    if x['Instructional Method'] == 'Online asynchronous' or \
    (x['Instructional Method'] == 'Online' and x['Course']=='PHYS 015'):
        color='green'
        linewidth=9
        
    counts = x['counts']
    
    # Skip over the independent studies or the plot is too busy
    if t != 'IS':
        continue

    if counts>1 and (x['Course'] in ['CSIS 200', 'PHYS 110', 'PHYS 142']):
        linewidth=2
        
    # Plot multiple versions if more than one class was taught
    # Get the numerical value of the y-points text
    pts = plt.plot([start,end],[y,y],fmt,linewidth=linewidth, color=color, markersize=1)

    # We only need to plot multiple sections for some classes. Others appear twice or more if
    # some students are taking it as a tutorial at the same time
    if (x['Course'] in ['CSIS 200', 'PHYS 110', 'PHYS 142']):
        if counts==2:
            ypt = pts[0].convert_yunits(y)[0]

            plt.plot([start,end],[ypt+0.25,ypt+0.25],fmt,linewidth=linewidth, color=color, markersize=1)

        elif counts==3:
            ypt = pts[0].convert_yunits(y)[0]

            plt.plot([start,end],[ypt+0.25,ypt+0.25],fmt,linewidth=linewidth, color=color, markersize=1)
            plt.plot([start,end],[ypt-0.25,ypt-0.25],fmt,linewidth=linewidth, color=color, markersize=1)
    
    
plt.xlim(pd.Timestamp('2012-01-01'), pd.Timestamp('2024-09-01')) 
#plt.xlim(pd.Timestamp('2012-01-01'), pd.Timestamp('2016-01-01')) 

plt.grid(axis='both')

# Sabbatical
plt.plot([pd.Timestamp('2018-06-15'),pd.Timestamp('2018-06-15')],[0,29],'k--')
plt.plot([pd.Timestamp('2019-09-01'),pd.Timestamp('2019-09-01')],[0,29],'k--')
plt.gca().text(pd.Timestamp('2019-01-01'), 15, r'SABBATICAL', fontsize=23, rotation=90)


###########################################################################################
# Custom legend
# https://matplotlib.org/stable/gallery/text_labels_and_annotations/custom_legends.html

custom_lines = [plt.Line2D([0], [0], color='blue', lw=8),
                plt.Line2D([0], [0], color='red', lw=8),
                plt.Line2D([0], [0], color='black', lw=8, linestyle=':'),
                plt.Line2D([0], [0], color='green', lw=12),
               ]
  
#plt.legend(custom_lines,['Lecture', 'Lab','Tutorial', 'Online asynchronous'], fontsize=18, loc='lower left')

#plt.ylim(-5)
plt.tight_layout()
plt.savefig('graph_of_independent_study_projects.png')

# Pretty print for LaTeX

In [ ]:
df

In [ ]:
output = ""

it = "\\it"
bf = "\\bf"
a0 = "{"
a1 = "}"

nrows = len(df)
for i in range(nrows):
    term = df.iloc[i]['Associated Term']
    course = df.iloc[i]['Course']
    name = df.iloc[i]['Course Title']
    
    if course.find('499')>=0 or course.find('497')>=0 or course.find('HNRS')>=0:
        continue
    
    t = df.iloc[i]['type']
    
    if t=='Lecture': 
        #output += f"{term:12s} &  {course}&  {name}\\\\\n"
        output += f"{term:12s} &  {a0}{bf} {course}{a1}&  {a0}{bf} {name}{a1}\\\\\n"
    elif t=='Lab':
        output += f"{term:12s} &  {a0}{it} {course}{a1}&  {a0}{it} {name}{a1}\\\\\n"
    elif t=='IS':
        output += f"{term:12s} &  {a0}{bf} {course}{a1}&  {a0}{bf} {name}{a1}\\\\\n"
    
    
print(output)
outfile = open("classes_for_table.tex",'w')
outfile.write(output)
outfile.close()


In [ ]:
!ls -ltr

# Scratch code

In [ ]:
#mask = df['Associated Term'] == 'Spring 2013'
mask = df['Associated Term'] == 'Spring 2021'
#mask = df['Associated Term'] == 'Spring 2017'

df[mask]

In [ ]:
mask = df['independent study'] != 'IS'

df[mask][['Associated Term', 'Course Title']].value_counts()

